<a href="https://colab.research.google.com/github/DasariRishikesh/Compression_NeuralNets/blob/main/ReSNET_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            # Compute scaled distillation loss from https://arxiv.org/abs/1503.02531
            # The magnitudes of the gradients produced by the soft targets scale
            # as 1/T^2, multiply them by T^2 when using both hard and soft targets.
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [ ]:
# Create the teacher


teacher = tf.keras.applications.ResNet101(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(32,32,3),
    pooling=None,
    classes=1000,
  

)



# Create the student
student = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(32,32,3),
    pooling=None,
    classes=1000,
    
)

# Clone student for later comparison
student_scratch = keras.models.clone_model(student)

In [ ]:
# Prepare the train and test dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normalize data
x_train = x_train.astype("float32") / 255.0
#x_train = np.reshape(x_train, (-1, 32, 32, 1))

x_test = x_test.astype("float32") / 255.0
#x_test = np.reshape(x_test, (-1, 32, 32, 1))

170498071/170498071 [==============================] - 14s 0us/step


In [ ]:
# Train teacher as usual
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate teacher on data.
teacher.fit(x_train, y_train, epochs=5)
teacher.evaluate(x_test, y_test)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1563/1563 [==============================] - 130s 71ms/step - loss: 2.2284 - sparse_categorical_accuracy: 0.3046
Epoch 2/5
1563/1563 [==============================] - 109s 70ms/step - loss: 1.9652 - sparse_categorical_accuracy: 0.3817
Epoch 3/5
1563/1563 [==============================] - 108s 69ms/step - loss: 1.8420 - sparse_categorical_accuracy: 0.4271
Epoch 4/5
1563/1563 [==============================] - 109s 70ms/step - loss: 2.1349 - sparse_categorical_accuracy: 0.2845
Epoch 5/5
313/313 [==============================] - 9s 24ms/step - loss: 2.8829 - sparse_categorical_accuracy: 0.2602


[2.882923126220703, 0.26019999384880066]

In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 105s 62ms/step - sparse_categorical_accuracy: 0.3222 - student_loss: 2.1918 - distillation_loss: 9.2934e-05
Epoch 2/5
1563/1563 [==============================] - 97s 62ms/step - sparse_categorical_accuracy: 0.4105 - student_loss: 1.8732 - distillation_loss: 1.0724e-04
Epoch 3/5
1563/1563 [==============================] - 109s 70ms/step - sparse_categorical_accuracy: 0.4109 - student_loss: 1.8250 - distillation_loss: 1.0358e-04
Epoch 4/5
1563/1563 [==============================] - 100s 64ms/step - sparse_categorical_accuracy: 0.3925 - student_loss: 1.8862 - distillation_loss: 1.0151e-04
Epoch 5/5
313/313 [==============================] - 5s 14ms/step - sparse_categorical_accuracy: 0.4131 - student_loss: 1.8042


[0.413100004196167, 1.3774927854537964]

In [ ]:
# Train student as done usually
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch.fit(x_train, y_train, epochs=5)
student_scratch.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 68s 40ms/step - loss: 2.2213 - sparse_categorical_accuracy: 0.3128
Epoch 2/5
1563/1563 [==============================] - 63s 40ms/step - loss: 1.9656 - sparse_categorical_accuracy: 0.3628
Epoch 3/5
1563/1563 [==============================] - 63s 41ms/step - loss: 1.7805 - sparse_categorical_accuracy: 0.4104
Epoch 4/5
1563/1563 [==============================] - 64s 41ms/step - loss: 1.7822 - sparse_categorical_accuracy: 0.4166
Epoch 5/5
313/313 [==============================] - 5s 14ms/step - loss: 2.0644 - sparse_categorical_accuracy: 0.4015


[2.064446449279785, 0.40149998664855957]

In [ ]:
!pip install -q tensorflow-addons
!pip install -q tensorflow-model-optimization

     |████████████████████████████████| 1.1 MB 31.2 MB/s 
     |████████████████████████████████| 238 kB 34.6 MB/s 


In [ ]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

%load_ext tensorboard

In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = x_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(teacher, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "resnet101"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 prune_low_magnitude_conv1_pad   (None, 38, 38, 3)   1           ['input_1[0][0]']                
 (PruneLowMagnitude)                                                                              
                                                                                                  
 prune_low_magnitude_conv1_conv  (None, 16, 16, 64)  18882       ['prune_low_magnitude_conv1_pad[0
  (PruneLowMagnitude)                                            ][0]']                           
                                                                                          

In [ ]:
_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(teacher, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Saved baseline model to: /tmp/tmpsa4wr2vy.h5


In [ ]:
 logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(x_train, y_train,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2
  6/352 [..............................] - ETA: 1:08 - loss: 1.8377 - accuracy: 0.3893

352/352 [==============================] - 160s 203ms/step - loss: 1.5500 - accuracy: 0.4453 - val_loss: 1.7111 - val_accuracy: 0.4228
Epoch 2/2
352/352 [==============================] - 66s 188ms/step - loss: 1.2365 - accuracy: 0.5534 - val_loss: 1.2264 - val_accuracy: 0.5696
